In [1]:
import pandas as pd
import numpy as np
from os import getcwd
from os.path import join

In [2]:
DIRECTORY = getcwd()
DIRECTORY

'C:\\Users\\conf\\Desktop\\agnia'

# Путь до директории, в которой лежит ноутбук, вычисляется самостоятельно
# Необходимо указать лишь имя файла (в переменной dataset_name)

In [3]:
dataset_name = "rdp_16s_v18.fa.dat"
dataset_path = join(DIRECTORY, dataset_name)

In [4]:
with open(dataset_path, encoding='utf-8', newline='\r\n') as f:
    line = f.readline().split('\t')
    columns_count, rows_count = int(line[0]), int(line[1])
    columns_type = [next(f).split() for _ in range(columns_count)]
column_to_type = dict(columns_type)

# 'Domain' - Домен
# 'Phylum' - Тип
# 'Class' - Класс
# 'Order' - Порядок
# 'Family' - Семейство
# 'Genus' - Род

In [5]:
headers = list(column_to_type.keys())

# Выбираем столбцы, по которым будет осуществляться группировка

In [6]:
groups_key = headers[2:5]
groups_key

['Phylum', 'Class', 'Order']

In [7]:
df = pd.read_csv(dataset_path, delimiter="\t", names=headers, skiprows=columns_count+1)

# Исходный датасет

In [8]:
df

,ID,Domain,Phylum,Class,Order,Family,Genus,AAA,AAC,AAG,...,TCT,TGA,TGC,TGG,TGT,TTA,TTC,TTG,TTT,CG
0,AJ000684,Bacteria,Actinobacteria,Actinobacteria,Mycobacteriales,Mycobacteriaceae,Mycobacterium,0.0104,0.0173,0.0208,...,0.0069,0.0125,0.0194,0.0284,0.0160,0.0076,0.0083,0.0132,0.0062,0.5859
1,EF599163,Bacteria,Proteobacteria,Gammaproteobacteria,Vibrionales,Vibrionaceae,Vibrio,0.0150,0.0176,0.0209,...,0.0052,0.0202,0.0183,0.0248,0.0144,0.0104,0.0078,0.0150,0.0065,0.5394
2,AY859683,Bacteria,Actinobacteria,Actinobacteria,Mycobacteriales,Mycobacteriaceae,Mycobacterium,0.0101,0.0176,0.0196,...,0.0054,0.0142,0.0176,0.0318,0.0149,0.0061,0.0095,0.0142,0.0054,0.5796
3,AY883036,Bacteria,Proteobacteria,Alphaproteobacteria,Rhodospirillales,Acetobacteraceae,Acetobacter,0.0095,0.0154,0.0212,...,0.0066,0.0220,0.0139,0.0308,0.0190,0.0088,0.0088,0.0147,0.0073,0.5545
4,DQ656489,Bacteria,Proteobacteria,Betaproteobacteria,Burkholderiales,Comamonadaceae,Aquincola,0.0152,0.0179,0.0214,...,0.0062,0.0186,0.0200,0.0255,0.0117,0.0097,0.0055,0.0131,0.0062,0.5686
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21190,NR_074339,Bacteria,Bacteroidetes,Cytophagia,Cytophagales,Cytophagaceae,Runella,0.0127,0.0161,0.0268,...,0.0054,0.0174,0.0188,0.0255,0.0114,0.0121,0.0074,0.0141,0.0107,0.5224
21191,NR_074270,Bacteria,Chlorobi,Chlorobia,Chlorobiales,Chlorobiaceae,Chloroherpeton,0.0162,0.0162,0.0325,...,0.0101,0.0217,0.0156,0.0250,0.0169,0.0129,0.0068,0.0156,0.0061,0.5115
21192,NR_074244,Bacteria,Proteobacteria,Alphaproteobacteria,Rhizobiales,Methylobacteriaceae,Methylobacterium,0.0102,0.0163,0.0197,...,0.0068,0.0183,0.0142,0.0299,0.0075,0.0102,0.0088,0.0129,0.0081,0.5752
21193,NR_074234,Bacteria,Bacteroidetes,Bacteroidia,Bacteroidales,Porphyromonadaceae,Porphyromonas,0.0136,0.0197,0.0204,...,0.0054,0.0244,0.0177,0.0143,0.0102,0.0136,0.0102,0.0163,0.0075,0.5302


<h1>Количество элементов в каждой группе организмов</h1>

In [9]:
groups = df.groupby(groups_key)['ID'].count().to_frame(name = 'count').reset_index()
groups

,Phylum,Class,Order,count
0,Abditibacteriota,Abditibacteria,Abditibacteriales,1
1,Acetothermia,Acetothermia_genera_incertae_sedis,_,1
2,Acidobacteria,Acidobacteria_Gp1,Acidicapsa,6
3,Acidobacteria,Acidobacteria_Gp1,Acidipila,2
4,Acidobacteria,Acidobacteria_Gp1,Acidobacterium,3
...,...,...,...,...
304,Woesearchaeota,Woesearchaeota_Incertae_Sedis_AR18,_,1
305,Woesearchaeota,Woesearchaeota_Incertae_Sedis_AR20,_,1
306,candidate_division_WPS-1,WPS-1_genera_incertae_sedis,_,2
307,candidate_division_WPS-2,WPS-2_genera_incertae_sedis,_,1


# Сохраняем группы в таблицу Excel

In [10]:
xlsx_name = "groups.xlsx"
xlsx_path = join(DIRECTORY, xlsx_name)
with pd.ExcelWriter(xlsx_path) as writer:
    groups.to_excel(writer, 
                    sheet_name = "AllGroups", 
                    index=False)

# Статистические характеристики групп организмов

In [11]:
groups_describe = groups.describe()
groups_describe

,count
count,309.000000
mean,68.592233
std,195.928564
min,1.000000
25%,2.000000
50%,7.000000
75%,31.000000
max,1782.000000


# По умолчанию мы обрезаем все группы, количество элементов в которых меньше среднего значения
## Если вы хотите выбрать другой параметр - измените значение переменной reduce_factor, расположенной ниже

In [12]:
reduce_factor = groups_describe.at['mean', 'count']
reduce_factor

68.59223300970874

# Количество элементов в оставшихся группах микроорганизмов

In [13]:
reduced_groups = df.groupby(groups_key)['ID'].count()[lambda x: x > reduce_factor].to_frame(name = 'count').reset_index()
reduced_groups

,Phylum,Class,Order,count
0,Actinobacteria,Actinobacteria,Actinomycetales,96
1,Actinobacteria,Actinobacteria,Bifidobacteriales,111
2,Actinobacteria,Actinobacteria,Micrococcales,940
3,Actinobacteria,Actinobacteria,Micromonosporales,297
4,Actinobacteria,Actinobacteria,Mycobacteriales,811
5,Actinobacteria,Actinobacteria,Propionibacteriales,286
6,Actinobacteria,Actinobacteria,Pseudonocardiales,401
7,Actinobacteria,Actinobacteria,Streptomycetales,1135
8,Actinobacteria,Actinobacteria,Streptosporangiales,414
9,Bacteroidetes,Bacteroidia,Bacteroidales,357


# Добавляем новую таблицу в excel-файл

In [14]:
with pd.ExcelWriter(xlsx_path, mode='a') as writer:
    reduced_groups.to_excel(writer, 
                            sheet_name = 'ReducedGroups', 
                            index=False)

# Статистические характеристики оставшихся групп организмов

In [15]:
reduced_groups_describe = reduced_groups.describe()
rg_min_value = reduced_groups_describe.at['min', 'count']
reduced_groups_describe

,count
count,49.000000
mean,373.469388
std,363.929543
min,71.000000
25%,105.000000
50%,254.000000
75%,466.000000
max,1782.000000


# Огранизмы, принадлежащие оставшимся группам

In [16]:
reduced_df = df.groupby(groups_key).filter(lambda group: len(group) > reduce_factor)
reduced_df

,ID,Domain,Phylum,Class,Order,Family,Genus,AAA,AAC,AAG,...,TCT,TGA,TGC,TGG,TGT,TTA,TTC,TTG,TTT,CG
0,AJ000684,Bacteria,Actinobacteria,Actinobacteria,Mycobacteriales,Mycobacteriaceae,Mycobacterium,0.0104,0.0173,0.0208,...,0.0069,0.0125,0.0194,0.0284,0.0160,0.0076,0.0083,0.0132,0.0062,0.5859
1,EF599163,Bacteria,Proteobacteria,Gammaproteobacteria,Vibrionales,Vibrionaceae,Vibrio,0.0150,0.0176,0.0209,...,0.0052,0.0202,0.0183,0.0248,0.0144,0.0104,0.0078,0.0150,0.0065,0.5394
2,AY859683,Bacteria,Actinobacteria,Actinobacteria,Mycobacteriales,Mycobacteriaceae,Mycobacterium,0.0101,0.0176,0.0196,...,0.0054,0.0142,0.0176,0.0318,0.0149,0.0061,0.0095,0.0142,0.0054,0.5796
3,AY883036,Bacteria,Proteobacteria,Alphaproteobacteria,Rhodospirillales,Acetobacteraceae,Acetobacter,0.0095,0.0154,0.0212,...,0.0066,0.0220,0.0139,0.0308,0.0190,0.0088,0.0088,0.0147,0.0073,0.5545
4,DQ656489,Bacteria,Proteobacteria,Betaproteobacteria,Burkholderiales,Comamonadaceae,Aquincola,0.0152,0.0179,0.0214,...,0.0062,0.0186,0.0200,0.0255,0.0117,0.0097,0.0055,0.0131,0.0062,0.5686
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21189,NR_074344,Bacteria,Actinobacteria,Actinobacteria,Mycobacteriales,Corynebacteriaceae,Corynebacterium,0.0107,0.0168,0.0195,...,0.0060,0.0141,0.0174,0.0322,0.0154,0.0087,0.0107,0.0154,0.0080,0.5680
21190,NR_074339,Bacteria,Bacteroidetes,Cytophagia,Cytophagales,Cytophagaceae,Runella,0.0127,0.0161,0.0268,...,0.0054,0.0174,0.0188,0.0255,0.0114,0.0121,0.0074,0.0141,0.0107,0.5224
21192,NR_074244,Bacteria,Proteobacteria,Alphaproteobacteria,Rhizobiales,Methylobacteriaceae,Methylobacterium,0.0102,0.0163,0.0197,...,0.0068,0.0183,0.0142,0.0299,0.0075,0.0102,0.0088,0.0129,0.0081,0.5752
21193,NR_074234,Bacteria,Bacteroidetes,Bacteroidia,Bacteroidales,Porphyromonadaceae,Porphyromonas,0.0136,0.0197,0.0204,...,0.0054,0.0244,0.0177,0.0143,0.0102,0.0136,0.0102,0.0163,0.0075,0.5302


In [17]:
reduced_df.set_index('ID')

,Domain,Phylum,Class,Order,Family,Genus,AAA,AAC,AAG,AAT,...,TCT,TGA,TGC,TGG,TGT,TTA,TTC,TTG,TTT,CG
ID,,,,,,,,,,,,,,,,,,,,,
AJ000684,Bacteria,Actinobacteria,Actinobacteria,Mycobacteriales,Mycobacteriaceae,Mycobacterium,0.0104,0.0173,0.0208,0.0111,...,0.0069,0.0125,0.0194,0.0284,0.0160,0.0076,0.0083,0.0132,0.0062,0.5859
EF599163,Bacteria,Proteobacteria,Gammaproteobacteria,Vibrionales,Vibrionaceae,Vibrio,0.0150,0.0176,0.0209,0.0189,...,0.0052,0.0202,0.0183,0.0248,0.0144,0.0104,0.0078,0.0150,0.0065,0.5394
AY859683,Bacteria,Actinobacteria,Actinobacteria,Mycobacteriales,Mycobacteriaceae,Mycobacterium,0.0101,0.0176,0.0196,0.0108,...,0.0054,0.0142,0.0176,0.0318,0.0149,0.0061,0.0095,0.0142,0.0054,0.5796
AY883036,Bacteria,Proteobacteria,Alphaproteobacteria,Rhodospirillales,Acetobacteraceae,Acetobacter,0.0095,0.0154,0.0212,0.0117,...,0.0066,0.0220,0.0139,0.0308,0.0190,0.0088,0.0088,0.0147,0.0073,0.5545
DQ656489,Bacteria,Proteobacteria,Betaproteobacteria,Burkholderiales,Comamonadaceae,Aquincola,0.0152,0.0179,0.0214,0.0152,...,0.0062,0.0186,0.0200,0.0255,0.0117,0.0097,0.0055,0.0131,0.0062,0.5686
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NR_074344,Bacteria,Actinobacteria,Actinobacteria,Mycobacteriales,Corynebacteriaceae,Corynebacterium,0.0107,0.0168,0.0195,0.0114,...,0.0060,0.0141,0.0174,0.0322,0.0154,0.0087,0.0107,0.0154,0.0080,0.5680
NR_074339,Bacteria,Bacteroidetes,Cytophagia,Cytophagales,Cytophagaceae,Runella,0.0127,0.0161,0.0268,0.0161,...,0.0054,0.0174,0.0188,0.0255,0.0114,0.0121,0.0074,0.0141,0.0107,0.5224
NR_074244,Bacteria,Proteobacteria,Alphaproteobacteria,Rhizobiales,Methylobacteriaceae,Methylobacterium,0.0102,0.0163,0.0197,0.0122,...,0.0068,0.0183,0.0142,0.0299,0.0075,0.0102,0.0088,0.0129,0.0081,0.5752


# Производим стратифицированную выборку

In [18]:
stratified_sampling = reduced_df.groupby(groups_key).apply(lambda group: group.sample(int(rg_min_value))).set_index('ID')
stratified_sampling

,Domain,Phylum,Class,Order,Family,Genus,AAA,AAC,AAG,AAT,...,TCT,TGA,TGC,TGG,TGT,TTA,TTC,TTG,TTT,CG
ID,,,,,,,,,,,,,,,,,,,,,
AJ427451,Bacteria,Actinobacteria,Actinobacteria,Actinomycetales,Actinomycetaceae,Schaalia,0.0094,0.0181,0.0174,0.0101,...,0.0080,0.0168,0.0188,0.0389,0.0121,0.0087,0.0087,0.0248,0.0094,0.5733
X80413,Bacteria,Actinobacteria,Actinobacteria,Actinomycetales,Actinomycetaceae,Schaalia,0.0106,0.0176,0.0176,0.0099,...,0.0085,0.0183,0.0183,0.0380,0.0127,0.0085,0.0070,0.0197,0.0070,0.5963
EF558367,Bacteria,Actinobacteria,Actinobacteria,Actinomycetales,Actinomycetaceae,Actinomyces,0.0081,0.0149,0.0162,0.0101,...,0.0088,0.0183,0.0176,0.0412,0.0142,0.0088,0.0095,0.0223,0.0156,0.5915
NR_113326,Bacteria,Actinobacteria,Actinobacteria,Actinomycetales,Actinomycetaceae,Actinomyces,0.0079,0.0171,0.0184,0.0092,...,0.0105,0.0184,0.0171,0.0375,0.0145,0.0072,0.0086,0.0184,0.0105,0.5913
JQ780830,Bacteria,Actinobacteria,Actinobacteria,Actinomycetales,Actinomycetaceae,Varibaculum,0.0106,0.0158,0.0196,0.0090,...,0.0128,0.0158,0.0219,0.0392,0.0203,0.0121,0.0090,0.0271,0.0181,0.5576
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AB680682,Bacteria,Tenericutes,Mollicutes,Mycoplasmoidales,Metamycoplasmataceae,Metamycoplasma,0.0173,0.0160,0.0243,0.0215,...,0.0104,0.0250,0.0153,0.0257,0.0125,0.0173,0.0090,0.0146,0.0083,0.4851
AF412989,Bacteria,Tenericutes,Mollicutes,Mycoplasmoidales,Metamycoplasmataceae,Mycoplasmopsis,0.0248,0.0227,0.0227,0.0207,...,0.0103,0.0200,0.0131,0.0227,0.0131,0.0207,0.0083,0.0145,0.0076,0.4746
AF125592,Bacteria,Tenericutes,Mollicutes,Mycoplasmoidales,Metamycoplasmataceae,Metamycoplasma,0.0221,0.0152,0.0255,0.0207,...,0.0103,0.0241,0.0138,0.0262,0.0124,0.0172,0.0083,0.0131,0.0069,0.4880


# Сохраняем данные в формате csv

In [19]:
reset_sample = stratified_sampling.reset_index()
reset_sample

,ID,Domain,Phylum,Class,Order,Family,Genus,AAA,AAC,AAG,...,TCT,TGA,TGC,TGG,TGT,TTA,TTC,TTG,TTT,CG
0,AJ427451,Bacteria,Actinobacteria,Actinobacteria,Actinomycetales,Actinomycetaceae,Schaalia,0.0094,0.0181,0.0174,...,0.0080,0.0168,0.0188,0.0389,0.0121,0.0087,0.0087,0.0248,0.0094,0.5733
1,X80413,Bacteria,Actinobacteria,Actinobacteria,Actinomycetales,Actinomycetaceae,Schaalia,0.0106,0.0176,0.0176,...,0.0085,0.0183,0.0183,0.0380,0.0127,0.0085,0.0070,0.0197,0.0070,0.5963
2,EF558367,Bacteria,Actinobacteria,Actinobacteria,Actinomycetales,Actinomycetaceae,Actinomyces,0.0081,0.0149,0.0162,...,0.0088,0.0183,0.0176,0.0412,0.0142,0.0088,0.0095,0.0223,0.0156,0.5915
3,NR_113326,Bacteria,Actinobacteria,Actinobacteria,Actinomycetales,Actinomycetaceae,Actinomyces,0.0079,0.0171,0.0184,...,0.0105,0.0184,0.0171,0.0375,0.0145,0.0072,0.0086,0.0184,0.0105,0.5913
4,JQ780830,Bacteria,Actinobacteria,Actinobacteria,Actinomycetales,Actinomycetaceae,Varibaculum,0.0106,0.0158,0.0196,...,0.0128,0.0158,0.0219,0.0392,0.0203,0.0121,0.0090,0.0271,0.0181,0.5576
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3474,AB680682,Bacteria,Tenericutes,Mollicutes,Mycoplasmoidales,Metamycoplasmataceae,Metamycoplasma,0.0173,0.0160,0.0243,...,0.0104,0.0250,0.0153,0.0257,0.0125,0.0173,0.0090,0.0146,0.0083,0.4851
3475,AF412989,Bacteria,Tenericutes,Mollicutes,Mycoplasmoidales,Metamycoplasmataceae,Mycoplasmopsis,0.0248,0.0227,0.0227,...,0.0103,0.0200,0.0131,0.0227,0.0131,0.0207,0.0083,0.0145,0.0076,0.4746
3476,AF125592,Bacteria,Tenericutes,Mollicutes,Mycoplasmoidales,Metamycoplasmataceae,Metamycoplasma,0.0221,0.0152,0.0255,...,0.0103,0.0241,0.0138,0.0262,0.0124,0.0172,0.0083,0.0131,0.0069,0.4880
3477,AY714305,Bacteria,Tenericutes,Mollicutes,Mycoplasmoidales,Metamycoplasmataceae,Mycoplasmopsis,0.0223,0.0216,0.0216,...,0.0079,0.0223,0.0151,0.0209,0.0115,0.0223,0.0072,0.0166,0.0094,0.4831


In [20]:
reset_sample.to_csv('sampling_data.csv',
                           sep=';',
                           encoding='utf-8',
                           index=False)

# Сохраняем в формате dat 
## Выбор CR/LF обсуловлен программой для дальнейшей обработки этих данных

In [24]:
reduced_dat = "reduced_" + dataset_name
with open(reduced_dat, 'w', encoding='utf-8', newline='\r\n') as f:
    f.write(f"{columns_count}\t{rows_count}\n")
    for key in column_to_type:
        f.write(f"{key}\t{column_to_type[key]}\n")
        
reset_sample.to_csv(reduced_dat, sep='\t', mode='a', header=False, index=False, lineterminator='\r\n')

# Находим наименьшее из среднеквадратических отклонений и триплет, соответствующий этому значению

## Вычисляем отклонения

In [25]:
std_triplet = reset_sample.std(numeric_only=True)
std_triplet

AAA    0.003135
AAC    0.002053
AAG    0.002833
AAT    0.003257
ACA    0.002062
         ...   
TTA    0.003394
TTC    0.002036
TTG    0.002720
TTT    0.002846
CG     0.030294
Length: 65, dtype: float64

## Находим триплет с наименьшим среднеквадратическим отклонением

In [26]:
min_triplet_index = std_triplet.idxmin()
min_triplet_index

'TCA'

## Среднеквадратическое отклонение этого триплета

In [27]:
std_triplet[min_triplet_index]

0.0017425800348062973